# Data Pipeline

#### Working from existing pipeline from previous progress reports

## Part 1: Webscraping news articles

### Install packages

In [1]:
import html5lib
import bs4
import requests
from bs4 import BeautifulSoup
import urllib
import urllib.request

### Creating list of Vox articles to parse

In [11]:
vox_URLs = ['https://www.vox.com/future-perfect/406453/trump-tariffs-china-shock', 'https://www.vox.com/scotus/406277/supreme-court-donald-trump-law-firms-arrogant', 'https://www.vox.com/immigration/406697/trump-immigration-deportation-poll-public-opinion-economy-approval-border', 'https://www.vox.com/politics/406742/wisconsin-supreme-court-election-results-susan-crawford-brad-schimel-elon-musk-democrats', 'https://www.vox.com/politics/406606/cory-booker-marathon-speech-filibuster-senate-floor-briefly-explained-protest', 'https://www.vox.com/politics/406467/trump-tariffs-prices-liberation-day-economy', 'https://www.vox.com/politics/405748/democrats-leader-michelle-obama-savior-sanders-aoc-walz-biden-harris', 'https://www.vox.com/the-logoff-newsletter-trump/406531/trump-third-presidential-term-constitution-briefly-explained', 'https://www.vox.com/scotus/406464/supreme-court-catholic-charities-wisconsin-religion', 'https://www.vox.com/politics/406357/democrat-tea-party-anger-rage-base-bernie-aoc-schumer-primary', 'https://www.vox.com/today-explained-newsletter/406329/trump-economy-ukraine-russia-germany-france-tariffs', 'https://www.vox.com/politics/406336/fentanyl-china-canada-mexico-trump', 'https://www.vox.com/politics/406403/foreign-aid-doge-trump-rubio', 'https://www.vox.com/politics/406398/elon-musk-wisconsin-supreme-court-payment-lottery']
len(vox_URLs)

14

### Creating list of Fox articles to parse

In [13]:
fox_URLs = ['https://www.foxnews.com/politics/wisconsin-florida-elections-get-spin-treatment-dems-gop-look-ahead-2026-midterms', 'https://www.foxnews.com/politics/house-republicans-brush-off-stefaniks-trump-admin-upheaval-amid-high-stakes-florida-race', 'https://www.foxnews.com/politics/pence-group-lashes-out-trump-tariffs-ahead-liberation-day-event-tax-american-consumers', 'https://www.foxnews.com/politics/reporters-notebook-april-fools-capitol-hill', 'https://www.foxnews.com/politics/fbi-flooded-record-number-new-agent-applications-kash-patels-first-month-leading-bureau', 'https://www.foxnews.com/us/trump-administration-deports-100k-illegal-migrants-since-inauguration-report', 'https://www.foxnews.com/politics/booker-called-filibuster-abuse-power-years-before-setting-senate-speech-record', 'https://www.foxnews.com/media/doge-volunteer-credits-trump-unprecedented-effort-curb-jaw-dropping-social-security-fraud', 'https://www.foxnews.com/media/planned-parenthood-apologizes-inadvertently-gives-sexually-explicit-coloring-book-children', 'https://www.foxnews.com/politics/dem-senator-who-ditched-tesla-protest-musk-refuses-call-violence-cars-domestic-terrorism']
len(fox_URLs)

10

### Fox Parser

In [2]:
import html5lib
import bs4
import requests
from bs4 import BeautifulSoup
import time

headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}
fox_URL = "https://www.foxnews.com/politics/house-republicans-brush-off-stefaniks-trump-admin-upheaval-amid-high-stakes-florida-race"

for url in fox_URLs:
    fox_r = requests.get(url=fox_URL, headers=headers)
    print(fox_r.content[:1000])
    fox_soup = BeautifulSoup(fox_r.content, 'html5lib')
    time.sleep(10)
print(fox_soup.prettify())

b'<!doctype html>\n<html data-n-head-ssr lang="en" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D">\n  <head>\n    <title>GOP leaders dismiss elections anxiety over Stefanik seat, Florida races | Fox News</title><meta data-n-head="ssr" http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta data-n-head="ssr" data-hid="content-type" http-equiv="content-type" content="text/html; charset=UTF-8"><meta data-n-head="ssr" data-hid="x-dns-prefetch-control" http-equiv="x-dns-prefetch-control" content="on"><meta data-n-head="ssr" charset="UTF-8"><meta data-n-head="ssr" data-hid="viewport" name="viewport" content="width=device-width, minimum-scale=1.0, initial-scale=1.0"><meta data-n-head="ssr" name="msapplication-square70x70logo" content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-70x70.png"><meta data-n-head="ssr" name="msapplication-square150x150logo" content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-150x150.png"><meta data-

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
 <head>
  <title>
   GOP leaders dismiss elections anxiety over Stefanik seat, Florida races | Fox News
  </title>
  <meta content="IE=edge,chrome=1" data-n-head="ssr" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" data-hid="content-type" data-n-head="ssr" http-equiv="content-type"/>
  <meta content="on" data-hid="x-dns-prefetch-control" data-n-head="ssr" http-equiv="x-dns-prefetch-control"/>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="width=device-width, minimum-scale=1.0, initial-scale=1.0" data-hid="viewport" data-n-head="ssr" name="viewport"/>
  <meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-70x70.png" data-n-head="ssr" name="msapplication-square70x70logo"/>
  <meta content="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/mstile-150x150.png" data-n-head="ssr" name="msapplication-s

In [4]:
import json
fox_articles=[]
table = fox_soup.find('head')
for row in table.find('script', attrs = {'data-n-head':'ssr', 'type':'application/ld+json'}):
    to_string = json.loads(row)
    fox_article = {}
    fox_article['headline'] = to_string['headline']
    fox_article['text'] = to_string['articleBody']
    fox_articles.append(fox_article)
print(fox_articles)

[{'headline': "House Republicans brush off Stefanik's Trump admin upheaval amid high-stakes Florida race", 'text': 'House Republicans are brushing off reports of anxiety over three historically red congressional seats, as the special election for two of them looms just a day away. "Jimmy Patronis and Randy Fine are exactly who House Republicans need to join our team," House Majority Whip Tom Emmer, R-Minn., told Fox News Digital last week, referring to candidates running in Florida\'s special election Tuesday. Patronis is running for Florida\'s 1st Congressional District, which was vacated by former Rep. Matt Gaetz, R-Fla., and Fine is running for Florida 6th District, previously held by former Rep. Mike Waltz, R-Fla., who now serves as U.S. national security advisor. Republicans have conceded they have been far outraised in those districts, and despite their likely victory, Democrats\' over-performance has reportedly raised concerns about the GOP\'s fate in other areas of the country.

### Vox Parser

In [5]:

vox_URL = "https://www.vox.com/scotus/406277/supreme-court-donald-trump-law-firms-arrogant"
vox_r = requests.get(url=vox_URL, headers=headers)
vox_r.encoding='utf-8-sig'
#print(vox_r.encoding)#
print(vox_r.content[:1000])


b'<!DOCTYPE html><html lang="en-US"><meta name="bingbot" content="nocache"/><head><meta charSet="utf-8"/><meta name="twitter:card" content="summary_large_image"/><meta name="twitter:site" content="@voxdotcom"/><meta property="fb:app_id" content="549923288395304"/><meta property="og:site_name" content="Vox"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><meta name="apple-mobile-web-app-title" content="Vox"/><link rel="alternate" type="application/rss+xml" title="Vox" href="/rss/index.xml"/><title>Trump\xe2\x80\x99s most arrogant action threatens his alliance with the Supreme Court | Vox</title><meta name="robots" content="index,follow,max-image-preview:large"/><meta name="description" content="\xef\xbb\xbfMaybe targeting John Roberts\xe2\x80\x99s law school classmates is a bad idea?"/><meta property="og:title" content="The cockiest thing Trump has done so far"/><meta property="og:description" content="\xef\xbb\xbfMaybe targeting John Roberts\xe2\

In [8]:
vox_soup = BeautifulSoup(vox_r.content, 'html5lib')
print(vox_soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta content="nocache" name="bingbot"/>
  <meta charset="utf-8"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="@voxdotcom" name="twitter:site"/>
  <meta content="549923288395304" property="fb:app_id"/>
  <meta content="Vox" property="og:site_name"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="Vox" name="apple-mobile-web-app-title"/>
  <link href="/rss/index.xml" rel="alternate" title="Vox" type="application/rss+xml"/>
  <title>
   Trump’s most arrogant action threatens his alliance with the Supreme Court | Vox
  </title>
  <meta content="index,follow,max-image-preview:large" name="robots"/>
  <meta content="﻿Maybe targeting John Roberts’s law school classmates is a bad idea?" name="description"/>
  <meta content="The cockiest thing Trump has done so far" property="og:title"/>
  <meta content="﻿Maybe targeting John Roberts’s law school classmates 

In [7]:
vox_articles=[]
vox_table = vox_soup.find('div', attrs = {'class':'duet--article--lede duet--page-layout--standard-article'})
#print(type(vox_table))
for row in vox_table.find_next('div', attrs = {'class':'duet--article--article-body-component'}, recursive=False):
    #print(row)
    vox_article = {}
    vox_article['headline'] = vox_table.find('h1', attrs = {'class':'h74scy1 h74scy0 h74scy6 xkp0cga xkp0cg9'}).text
    vox_article['subheading'] = vox_table.find('p', attrs = {'class':'duet--article--dangerously-set-cms-markup h74scyh h74scyg xkp0cga'}).text
    vox_article['text'] = [i.text for i in vox_table.find_all_next('p', attrs = {'class':'duet--article--dangerously-set-cms-markup duet--article--standard-paragraph _1agbrixi lg8ac51 lg8ac50 xkp0cg1'})]
    vox_articles.append(vox_article)
print(len(vox_article['text']), vox_articles)


29 [{'headline': 'The cockiest thing Trump has done so far', 'subheading': '\ufeffMaybe targeting John Roberts’s law school classmates is a bad idea?', 'text': ['All nine of the Supreme Court justices are lawyers. All of them have friends and law school classmates in private practice. All of them sit at the apex of a legal system that depends on lawyers to brief judges on the matters those judges must decide. Many of them were themselves litigators at large law firms, where their livelihood depended on their ability to advocate for their clients without fear of personal reprisals. ', 'So it’s hard to imagine a presidential action that is more likely to antagonize the justices President Donald Trump needs to uphold his agenda, not to mention every other federal judge who isn’t already in the tank for MAGA, than a series of executive orders Trump has recently issued. These actions aim to punish law firms that previously represented Democrats or clients opposed to Trump. ', 'The lawyers t

The below functions for extracting news data from Fox News were heavily based on an existing Fox News webscraper published on GitHub by the user StickmanNinja. The license for the scraper is Apache 2.0. I have adjusted the below functions to make them conducive to Python 3.0 (original scraper appears to have been made using an earlier version of Python), and have changed several of the variable names.

Here is a link to the repository for the webscraper: [Fox-News-Web-Scraper](https://github.com/StickmanNinja/Fox-News-Web-Scraper/tree/master)

In [ ]:
#https://github.com/StickmanNinja/Fox-News-Web-Scraper/blob/master/FoxNewsScraper.py
def title_url(fox_art):
    articlesList = []
    for text in fox_art:
        print(text)
        tag = text.find("a")
        #tag = text.find("h3", {'class':'title'})
        print(tag)
        title = tag.getText()
        url = tag.get("href")
        artDict = {"title" : title, "url" : url}
        articlesList.append(artDict)

def fox_parser():
    URL = "http://www.foxnews.com/"
    webpage = urllib.request.urlopen(URL)
    r  = requests.get(URL)
    art_text = r.text
    soup = BeautifulSoup(art_text,"lxml")
    for art in range(0, 100):
        fox_articles = soup.find_all("article", class_ ="article story-" + str(art))
        print(fox_articles)
        title_url(fox_articles)
        
fox_parser()
title_url()

### Parse contents

In [ ]:
soup = BeautifulSoup(r.content, 'html5lib')
#print(soup.prettify())

In [ ]:
#soup.find_all("script")[:1000]

In [ ]:
#soup.find('data-n-head'=="ssr")

In [ ]:
#headline_text=[]
#table = soup.find('script')
#for row in table.find_all('script', attrs = {'data-n-head':'ssr', 'type': 'application/ld+json'}):
    #return(row)
    #headline = row.find('headline').text.strip()
    #text = row.find('articleBody').text.strip()
    #title_text.append({'headline': headline, 'text':text})

In [ ]:
#headline_text

## Part 2: Uploading and parsing subjective intensifiers data

#### Import dependencies

In [ ]:
import numpy as np
import pandas as pd
import re

#### Read in data

In [ ]:
#https://stackoverflow.com/questions/64966365/parse-sgml-using-beautiful-soup
with open("acl-99-data/subjectivity.data", "r") as f:
    entries = f.read()
    soup = BeautifulSoup(entries, "html.parser")
print(soup)
type(soup)

In [ ]:
sublists = []
data_str = str(soup)
data_list = data_str.split('<mc')
for line in data_list:
    line = line.split(' ')
    sublists.append(line)

for line2 in sublists:
    del line2[0]

for lines in sublists:
    if len(lines) > 10:
        print(lines)

In [ ]:
#make list of dictionaries for each word
listDict = []
for sublist in range(len(sublists)):
    listDict.append({})
len(listDict)

#each dictionary is a word
attr_list = []
for sublist in sublists:
    for items in sublist:
        item = items.split('=')
        attr_list.append(item)

attr_List = []
group = []
for item in attr_list:
    if item[0] == 'cluster':
        attr_List.append(group)
        group = []
    group.append(item)

for item in items:
    for diction in listDict:
    #print(item[0])
        if item[0] == 'cluster':
            diction['cluster'] = item[1]

attr_List

#### Creating dataframe from subjective intensifiers data

#### Basic assessment of data